In [ ]:
from IPython.display import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold, RepeatedKFold, LeaveOneOut, LeavePOut, ShuffleSplit, StratifiedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import learning_curve, validation_curve
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('./banana_quality.csv')
data.head()

# Разделим датасет на тестовую и обучающую выборки

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
X = data.drop('Quality', axis=1)
y = data['Quality']#ключевое значени
label_encoder = LabelEncoder()
y = data['Quality'] = label_encoder.fit_transform(data['Quality'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def class_proportions(array: np.ndarray) :
    """
    Вычисляет пропорции классов
    array - массив, содержащий метки классов
    """
    # Получение меток классов и количества меток каждого класса
    labels, counts = np.unique(array, return_counts=True)
    # Превращаем количество меток в процент их встречаемости
    # делим количество меток каждого класса на общее количество меток
    counts_perc = counts/array.size
    # Теперь sum(counts_perc)==1.0
    # Создаем результирующий словарь,
    # ключом словаря явлется метка класса,
    # а значением словаря процент встречаемости метки
    res = dict()
    for label, count2 in zip(labels, zip(counts, counts_perc)):
        res[label] = count2
    return res

def print_class_proportions(array: np.ndarray):
    """
    Вывод пропорций классов
    """
    proportions = class_proportions(array)
    if len(proportions)>0:
        print('Метка \t Количество \t Процент встречаемости')
    for i in proportions:
        val, val_perc = proportions[i]
        val_perc_100 = round(val_perc * 100, 2)
        print('{} \t {} \t \t {}%'.format(i, val, val_perc_100))

In [ ]:
print_class_proportions(y_test)

In [ ]:
print_class_proportions(y_train)

# Построим модель на основе метода ближайших соседей для К=2 и К=10

In [ ]:
# 2 ближайших соседа
cl1_1 = KNeighborsClassifier(n_neighbors=2)
cl1_1.fit(X_train, y_train)
target1_1 = cl1_1.predict(X_test)
len(target1_1), target1_1

In [ ]:
# 10 ближайших соседей
cl1_2 = KNeighborsClassifier(n_neighbors=10)
cl1_2.fit(X_train, y_train)
target1_2 = cl1_2.predict(X_test)
len(target1_2), target1_2

# Оценка качества моделей (произвольной и оптимальной)

## Метрика Accuracy

In [ ]:
balanced_accuracy_score(y_test, target1_1)#Для модели с К=2

In [ ]:
balanced_accuracy_score(y_test, target1_2)#Для модели с К=10

## Матрица ошибок

In [ ]:
#Для К=2
tn, fp, fn, tp = confusion_matrix(y_test, target1_1).ravel()
tn, fp, fn, tp

In [ ]:
#Для К=10
tn, fp, fn, tp = confusion_matrix(y_test, target1_2).ravel()
tn, fp, fn, tp

## Метрика precision

Доля верно предсказанных классификатором положительных объектов, из всех объектов, которые классификатор верно или неверно определил как положительные.

In [ ]:
#Для К=2
precision_score(y_test, target1_1)

In [ ]:
#Для К=10
precision_score(y_test, target1_2)

## Метрика recall

Доля верно предсказанных классификатором положительных объектов, из всех действительно положительных объектов.



In [ ]:
#Для К=2
recall_score(y_test, target1_1)

In [ ]:
#Для К=10
recall_score(y_test, target1_2)

## F-мера

Для того, чтобы объединить precision и recall в единую метрику используется Fβ
-мера, которая вычисляется как среднее гармоническое от precision и recall:

In [ ]:
#Для К=2
f1_score(y_test,target1_1)

In [ ]:
#Для К=10
f1_score(y_test,target1_2)

Выведем метрики для каждого из классов

In [ ]:
#Для К=2
classification_report(y_test, target1_1,
                      target_names=["Bad", "Good"], output_dict=True)

In [ ]:
#Для К=10
classification_report(y_test, target1_1,
                      target_names=["Bad", "Good"], output_dict=True)

## ROC-кривая и ROC AUC

In [ ]:
def draw_roc_curve(y_true, y_score, pos_label, average):
    fpr, tpr, thresholds = roc_curve(y_true, y_score,
                                     pos_label=pos_label)
    roc_auc_value = roc_auc_score(y_true, y_score, average=average)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc_value)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Для 2 ближайших соседей
draw_roc_curve(y_test, target1_1, pos_label=1, average='micro')

In [ ]:
#Для 10 ближайших соседей
draw_roc_curve(y_test, target1_2, pos_label=1, average='micro')

# Кросс-валидация

## Автоматический выбор стратегии кросс-валидации

In [ ]:
scoring = {'precision': 'precision_weighted',
           'recall': 'recall_weighted',
           'f1': 'f1_weighted'}
scores = cross_validate(KNeighborsClassifier(n_neighbors=2),
                        data, y, scoring=scoring,
                        cv=3, return_train_score=True)
scores

##Стратегия Kfold

In [ ]:
kf = KFold(n_splits=5)
scores = cross_val_score(KNeighborsClassifier(n_neighbors=2),
                         data, y, scoring='f1_weighted',
                         cv=kf)
scores

In [ ]:
kf = KFold(n_splits=5)
scores = cross_validate(KNeighborsClassifier(n_neighbors=2),
                        data, y, scoring=scoring,
                        cv=kf, return_train_score=True)
scores

##Стратегия Repeated K-Fold

In [ ]:
kf = RepeatedKFold(n_splits=5, n_repeats=3)
scores = cross_validate(KNeighborsClassifier(n_neighbors=2),
                        data, y, scoring=scoring,
                        cv=kf, return_train_score=True)
scores

## Стратегия Leave One Out (LРO) и стратегия Leave P Out

не эффективны на большой выборке

##Стратегия ShuffleSplit

In [ ]:
kf = ShuffleSplit(n_splits=5, test_size=0.25)
scores = cross_validate(KNeighborsClassifier(n_neighbors=2),
                        data, y, scoring=scoring,
                        cv=kf, return_train_score=True)
scores

# Оптимизация

## GridSearch

In [ ]:
n_range = np.array(range(5,55,5))
tuned_parameters = [{'n_neighbors': n_range}]
tuned_parameters

In [ ]:
%%time
clf_gs = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring='accuracy')
clf_gs.fit(X_train, y_train)

In [ ]:
clf_gs.cv_results_

In [ ]:
# Лучшее значение метрики и лучшее значение параметров
(clf_gs.best_score_, clf_gs.best_params_)

In [ ]:
# Изменение качества на тестовой выборке в зависимости от К-соседей
plt.plot(n_range, clf_gs.cv_results_['mean_test_score'])

## RandomizedSearch

In [ ]:
%%time
clf_rs = RandomizedSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring='accuracy')
clf_rs.fit(X_train, y_train)

In [ ]:
clf_rs.best_score_, clf_rs.best_params_

In [ ]:
# Изменение качества на тестовой выборке в зависимости от К-соседей
plt.plot(n_range, clf_rs.cv_results_['mean_test_score'])